**Step 1:** Mount your Google Drive by clicking on "Mount Drive" in the Files section (panel to the left of this text.)

**Step 2:** Go to Runtime -> Change runtime type and select TPU.

**Step 3:** Create a folder in your Google Drive, and rename it to "vMalConv"

**Step 4:** Download the pre-processed training and test datasets.

In [ ]:
!wget https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/X_test.dat
!wget https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/X_train.dat
!wget https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/y_train.dat
!wget https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/y_test.dat
!wget https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/metadata.csv

--2021-10-13 20:19:00--  https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/X_test.dat
Resolving dsci6015aisecf21.s3.us-east-2.amazonaws.com (dsci6015aisecf21.s3.us-east-2.amazonaws.com)... 52.219.99.82
Connecting to dsci6015aisecf21.s3.us-east-2.amazonaws.com (dsci6015aisecf21.s3.us-east-2.amazonaws.com)|52.219.99.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1904800000 (1.8G) [application/x-www-form-urlencoded]
Saving to: ‘X_test.dat’

X_test.dat          100%[===================>]   1.77G   108MB/s    in 33s     

2021-10-13 20:19:33 (55.7 MB/s) - ‘X_test.dat’ saved [1904800000/1904800000]

--2021-10-13 20:19:33--  https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/X_train.dat
Resolving dsci6015aisecf21.s3.us-east-2.amazonaws.com (dsci6015aisecf21.s3.us-east-2.amazonaws.com)... 52.219.107.50
Connecting to dsci6015aisecf21.s3.us-east-2.amazonaws.com (dsci6015aisecf21.s3.us-east-2.amazonaws.com)|52.219.107.50|:443... connected.
HTTP request sent, await

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Step 5:** Copy the downloaded files to vMalConv

In [ ]:
!cp /content/X_train.dat /content/drive/MyDrive/vMalConv/X_train.dat
!cp /content/X_test.dat /content/drive/MyDrive/vMalConv/X_test.dat
!cp /content/y_train.dat /content/drive/MyDrive/vMalConv/y_train.dat
!cp /content/y_test.dat /content/drive/MyDrive/vMalConv/y_test.dat
!cp /content/metadata.csv /content/drive/MyDrive/vMalConv/metadata.csv

**Step 6:** Download and install Ember:

In [ ]:
!wget https://github.com/endgameinc/ember/archive/master.zip
!unzip master.zip
!rm master.zip
!cp -r ember-master/* .
!rm -r ember-master
!pip install -r requirements.txt
!python setup.py install

--2021-10-13 20:24:57--  https://github.com/endgameinc/ember/archive/master.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/elastic/ember/archive/master.zip [following]
--2021-10-13 20:24:57--  https://github.com/elastic/ember/archive/master.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/elastic/ember/zip/master [following]
--2021-10-13 20:24:57--  https://codeload.github.com/elastic/ember/zip/master
Resolving codeload.github.com (codeload.github.com)... 140.82.112.10
Connecting to codeload.github.com (codeload.github.com)|140.82.112.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [           <=>      ]  11.22M  4.17MB/s    i

**Step 7:** Read vectorized features from the data files.

In [ ]:
import ember
X_train, y_train, X_test, y_test = ember.read_vectorized_features("drive/MyDrive/vMalConv/")
metadata_dataframe = ember.read_metadata("drive/MyDrive/vMalConv/")

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


**Step 7:** Get rid of rows with no labels.

In [ ]:
labelrows = (y_train != -1)
X_train = X_train[labelrows]
y_train = y_train[labelrows]

In [ ]:
import h5py
h5f = h5py.File('X_train.h5', 'w')
h5f.create_dataset('X_train', data=X_train)
h5f.close()
h5f = h5py.File('y_train.h5', 'w')
h5f.create_dataset('y_train', data=y_train)
h5f.close()

In [ ]:
!cp /content/X_train.h5 /content/drive/MyDrive/vMalConv/X_train.h5
!cp /content/y_train.h5 /content/drive/MyDrive/vMalConv/y_train.h5

> **Exercise 1:** Complete the following code to create the architecture of MalConv in Keras:

In [ ]:
def make_model():
  import tensorflow as tf
  from tensorflow import keras
  from tensorflow.keras import layers
  feature_size=2381
  tf.compat.v1.disable_eager_execution()

  keras.backend.clear_session()
  
  # Model architecture
  from tensorflow.keras import layers
  
 ### Your code -- Define the layers of MalConv ###
  inp = Input( shape=(maxlen,))
  emb = Embedding( input_dim, embedding_size )( inp )
  filt = Conv1D( filters=128, kernel_size=500, strides=500, use_bias=True, activation='relu', padding='valid' )(emb)
  attn = Conv1D( filters=128, kernel_size=500, strides=500, use_bias=True, activation='sigmoid', padding='valid')(emb)
  gated = Multiply()([filt,attn])
  feat = GlobalMaxPooling1D()( gated )
  dense = Dense(128, activation='relu')(feat)
  outp = Dense(1, activation='sigmoid')(dense)
  model.compile(tf.keras.optimizers.Adam(learning_rate=0.001),
  loss='binary_crossentropy',
  metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision()])
  print(model.summary())
  return model

In [ ]:
#Function for the model
def myModel():

    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers
    from tensorflow.keras.models import Sequential
    from keras import regularizers
    tf.compat.v1.disable_eager_execution()
    
    #Model architecture
    model = Sequential()
    model.add(layers.InputLayer(input_shape=(2381,))) 
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(units = 1000, activation = tf.nn.relu, activity_regularizer=regularizers.l2(0.01)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(units = 1, activation=tf.nn.sigmoid))
    print(model.summary())
    
    #model compilation
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    model.save('my_model.h5')
    
    return model

In [ ]:
model = myModel()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_2 (Dropout)          (None, 2381)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              2382000   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 1001      
Total params: 2,383,001
Trainable params: 2,383,001
Non-trainable params: 0
_________________________________________________________________
None


**Step 8:** Partial fit the standardScaler to avoid overloading the memory:

In [ ]:
from sklearn.preprocessing import StandardScaler
mms = StandardScaler()
for x in range(0,600000,100000):
  mms.partial_fit(X_train[x:x+100000])

In [ ]:
X_train = mms.transform(X_train)

In [ ]:
X_train.shape

(600000, 2381)

In [ ]:
## Reshape to create 3 channels ##
#import numpy as np
#X_train = np.reshape(X_train,(-1,1,2381))
#y_train = np.reshape(y_train,(-1,1,1))

> **Exercise 2:** Complete the following code to train the model for 30 epochs, with a batch size of 128, and 20% validation split. 

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

save_dir = "drive/MyDrive/vMalConv/"

#callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

### Your code ###

history = model.fit(X_train, y_train,batch_size=256,epochs=30,validation_split=.2)
# Save the weights #
model.save_weights (save_dir+'weights.h5')

# Save the model architecture #
model_json = model.to_json()
with open(save_dir+"model.json", "w") as json_file:
    json_file.write(model_json)

print("model saved.")

Train on 480000 samples, validate on 120000 samples
Epoch 1/30
479232/480000 [============================>.] - ETA: 0s - loss: 51.2133 - accuracy: 0.9364

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


480000/480000 [==============================] - 31s 66us/sample - loss: 51.1332 - accuracy: 0.9364 - val_loss: 30.9278 - val_accuracy: 0.9478
Epoch 2/30
480000/480000 [==============================] - 33s 70us/sample - loss: 39.9438 - accuracy: 0.9338 - val_loss: 23.0442 - val_accuracy: 0.9712
Epoch 3/30
480000/480000 [==============================] - 30s 63us/sample - loss: 32.8642 - accuracy: 0.9353 - val_loss: 16.5891 - val_accuracy: 0.9741
Epoch 4/30
480000/480000 [==============================] - 36s 74us/sample - loss: 25.9813 - accuracy: 0.9377 - val_loss: 14.1714 - val_accuracy: 0.9739
Epoch 5/30
480000/480000 [==============================] - 33s 69us/sample - loss: 24.9268 - accuracy: 0.9366 - val_loss: 13.1503 - val_accuracy: 0.9728
Epoch 6/30
480000/480000 [==============================] - 30s 63us/sample - loss: 6.6028 - accuracy: 0.9370 - val_loss: 11.7890 - val_accuracy: 0.9755
Epoch 7/30
480000/480000 [==============================] - 27s 57us/sample - loss: 18.8

**Final Steps:** Download this Jupyter Notebook, and the saved model files.